In [40]:
import os

import dspy
import httpx
from bs4 import BeautifulSoup
from dotenv import load_dotenv

load_dotenv()

True

In [41]:
API_BASE = os.getenv("LITELLM_URL")
API_KEY = os.getenv("LITELLM_API_KEY")

In [42]:
lm = dspy.LM('azure/gpt-4.1-nano-sweden', temperature=0, api_base=API_BASE, api_key=API_KEY)
dspy.configure(lm=lm)

In [65]:
def get_trending_repositories() -> list[str]:
    """Returns a top daily trending repositories on GitHub."""

    html = httpx.get("https://github.com/trending")
    soup = BeautifulSoup(html.text, "html.parser")
    soup = soup.find_all("article", class_="Box-row")

    descriptions = [repo.text.replace("\n", "") for repo in soup]
    return descriptions

def get_repository_details(profile: str, name: str) -> str:
    """Returns the README of the GitHub repository, if it exists."""

    html = httpx.get(f"https://raw.githubusercontent.com/{profile}/{name}/refs/heads/main/README.md")
    return html.text

In [69]:
instructions = "Find all relevant GitHub repositories to satisfy query."
signature = dspy.Signature("query -> answer: str", instructions)
agent = dspy.ReAct(signature, tools=[get_trending_repositories, get_repository_details], max_iters=5)

In [70]:
response = agent(query="I want to learn about new existing repositories in the field of AI. What are the trending repositories?")

In [71]:
response.trajectory

{'thought_0': 'Since the user is interested in trending repositories related to AI, I will first retrieve the current top trending repositories on GitHub to identify relevant projects in the AI field.',
 'tool_name_0': 'get_trending_repositories',
 'tool_args_0': {},
 'observation_0': ['           Star        microsoft /      qlib       Qlib is an AI-oriented quantitative investment platform that aims to realize the potential, empower research, and create value using AI technologies in quantitative investment, from exploring ideas to implementing productions. Qlib supports diverse machine learning modeling paradigms. including supervised learning, market dynamics modeling, and RL.    Python        19,996        3,257        Built by                  333 stars today ',
  '           Star        All-Hands-AI /      OpenHands       🙌 OpenHands: Code Less, Make More    Python        55,449        6,232        Built by                  533 stars today ',
  '           Star        modelconte

In [72]:
print(response.answer)

The trending repositories in the field of AI include:
1. **Qlib** by Microsoft: An open-source, AI-oriented quantitative investment platform supporting diverse machine learning paradigms, financial data analysis, and automated trading workflows. It is actively developed and widely used in quantitative research.
2. **MindsDB**: An AI data solution that allows querying data in natural language and SQL, integrating data from various sources, and building AI knowledge bases. It is designed for easy deployment and powerful data management with AI capabilities.

These repositories are currently among the most popular and relevant in AI, reflecting the latest trends and active development in the field.
